# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import json
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files('data/song_data')

In [5]:
filepath = song_files[0]

In [6]:
print(filepath)


#with open(filepath) as f:
 # data = json.load(f)

# Output: {'name': 'Bob', 'languages': ['English', 'Fench']}
#print(data)

#ser = pd.read_json(filepath, typ='series')
df = pd.DataFrame([pd.read_json(filepath,  typ='series')])
df.head()
print(df)
print(df.columns)
print(df.head())
print(df.values[0][1])

/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json
   num_songs           artist_id artist_latitude artist_longitude  \
0          1  ARD7TVE1187B99BFB1            None             None   

   artist_location artist_name             song_id             title  \
0  California - LA      Casual  SOMZWCG12A8C13C480  I Didn't Mean To   

    duration  year  
0  218.93179     0  
Index(['num_songs', 'artist_id', 'artist_latitude', 'artist_longitude',
       'artist_location', 'artist_name', 'song_id', 'title', 'duration',
       'year'],
      dtype='object')
   num_songs           artist_id artist_latitude artist_longitude  \
0          1  ARD7TVE1187B99BFB1            None             None   

   artist_location artist_name             song_id             title  \
0  California - LA      Casual  SOMZWCG12A8C13C480  I Didn't Mean To   

    duration  year  
0  218.93179     0  
ARD7TVE1187B99BFB1


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_data = [df.values[0][6], df.values[0][7], df.values[0][1], df.values[0][9], df.values[0][8]]
print(song_data)
#song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
print(song_table_insert)
print(song_data)

cur.execute(song_table_insert, song_data)
conn.commit()

INSERT INTO songs (song_id, title, artist_id, year, duration)                                 VALUES (%s, %s, %s, %s, %s) ON CONFLICT DO NOTHING 
['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data = [df.values[0][1], df.values[0][5], df.values[0][4], df.values[0][2], df.values[0][3]]
print(artist_data)
#artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', None, None]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = get_files('data/log_data')

In [12]:
filepath = log_files[0]

In [13]:
df = pd.DataFrame(pd.read_json(filepath,  lines=True))
#df.head()
print(df.columns)
print(df.values[0])
print(df.values[1])
print(len(df.index))

Index(['artist', 'auth', 'firstName', 'gender', 'itemInSession', 'lastName',
       'length', 'level', 'location', 'method', 'page', 'registration',
       'sessionId', 'song', 'status', 'ts', 'userAgent', 'userId'],
      dtype='object')
['Stephen Lynch' 'Logged In' 'Jayden' 'M' 0 'Bell' 182.85669 'free'
 'Dallas-Fort Worth-Arlington, TX' 'PUT' 'NextSong' 1540991795796.0 829
 "Jim Henson's Dead" 200 1543537327796
 'Mozilla/5.0 (compatible; MSIE 10.0; Windows NT 6.2; WOW64; Trident/6.0)'
 '91']
['Manowar' 'Logged In' 'Jacob' 'M' 0 'Klein' 247.562 'paid'
 'Tampa-St. Petersburg-Clearwater, FL' 'PUT' 'NextSong' 1540558108796.0
 1049 'Shell Shock' 200 1543540121796
 '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.78.2 (KHTML, like Gecko) Version/7.0.6 Safari/537.78.2"'
 '73']
388


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
df = df[(df["page"] == "NextSong")]

print(len(df.index))
#df2.head()
#Date_Time = pd.to_datetime(df2.ts, unit='ms')


330


In [15]:
t = pd.to_datetime(df.ts, unit='ms')
t.head()

print(t[0])
print(t[0].hour, t[0].day, t[0].week , t[0].month, t[0].year, t[0].weekday())
print(df.ts.head())

2018-11-30 00:22:07.796000
0 30 48 11 2018 4
0    1543537327796
1    1543540121796
2    1543540368796
3    1543540625796
4    1543540856796
Name: ts, dtype: int64


In [16]:
# timestamp, hour, day, week of year, month, year, and weekday
time_data = [t, t.dt.hour, t.dt.day, t.dt.week , t.dt.month, t.dt.year, t.dt.weekday]
column_labels = ('timestamp', 'hour', 'day', 'week of year', 'month', 'year', 'weekday')

In [17]:
time_df = pd.DataFrame({c: d for c,d in zip (column_labels, time_data)}).dropna()
#time_df = pd.DataFrame(time_data, column_labels)
#time_df = time_df.transpose()
time_df.head()

,timestamp,hour,day,week of year,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_data = [df.userId, df.firstName, df.lastName, df.gender, df.level]
column_labels_user = ('userId', 'firstName', 'lastName', 'gender', 'level')
user_df = pd.DataFrame(user_data, column_labels_user)
user_df = user_df.transpose()
user_df = user_df.drop_duplicates(subset=['userId'])
#filter = user_df['userId'] != ""
#user_df = user_df[filter]
user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [22]:
for index, row in df.iterrows():
    
    #newselect = ("""SELECT songs.song_id, artists.artist_id FROM (artists JOIN songs ON artists.artist_id = songs.artist_id) \
     #                           WHERE songs.title = %s AND artists.name = %s AND songs.duration = %s; """)
    
    #newselect = ("""SELECT songs.song_id, artists.artist_id FROM (artists JOIN songs ON artists.artist_id = songs.artist_id) \
     #                          WHERE songs.title = $$%s$$  AND artists.name = $$%s$$; """)
    
    #newselect = ("""SELECT songs.song_id, artists.artist_id FROM (artists JOIN songs ON artists.artist_id = songs.artist_id) \
     #                          WHERE songs.title = $$Jim Henson's Dead$$ AND artists.name = $$Stephen Lynch$$; """)
    
    #newselect = ("""SELECT songs.song_id, artists.artist_id FROM (SELECT song_id FROM songs WHERE title = $$Jim Henson's Dead$$ \
    #AND artist_id = (SELECT artist_id FROM artists WHERE name = $$Stephen Lynch$$)); """)
   
    
    

    # get songid and artistid from song and artist tables
    #cur.execute(newselect)
    print(row.song, row.artist)
    cur.execute(song_select, (row.song, row.artist))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
    print(songid, artistid)    

Jim Henson's Dead Stephen Lynch


IndexError: tuple index out of range

In [21]:
for index, row in df.iterrows():
    
    newselect = ("""SELECT songs.song_id, artists.artist_id FROM (artists JOIN songs ON artists.artist_id = songs.artist_id) \
                                WHERE songs.title = '%s' AND artists.name = %s AND songs.duration = %s; """)
    
    #newselect = ("""SELECT songs.song_id, artists.artist_id FROM (artists JOIN songs ON artists.artist_id = songs.artist_id) \
     #                           WHERE songs.title = Jim Henson's Dead AND artists.name = 'Stephen Lynch' AND songs.duration = 182.85669; """)

    print(row.song, row.artist, row.length)
    # get songid and artistid from song and artist tables
    #cur.execute(newselect)
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    print(songid, artistid)    
    songplay_data = [row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent]
    print("----------------")
    # insert songplay record
    #songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Jim Henson's Dead Stephen Lynch 182.85669
None None
----------------


IndexError: list index out of range

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.